In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pylab
from google.colab.patches import cv2_imshow
from moviepy.editor import VideoFileClip
from moviepy import *
%matplotlib inline
import time
import os
import glob

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2605056/45929032 bytes (5.7%)5677056/45929032 bytes (12.4%)8536064/45929032 bytes (18.6%)11550720/45929032 bytes (25.1%)14155776/45929032 bytes (30.8%)16850944/45929032 bytes (36.7%)19652608/45929032 bytes (42.8%)22650880/45929032 bytes (49.3%)25665536/45929032 bytes (55.9%)28696576/45929032 bytes (62.5%)31596544/45929032 bytes (68.8%)34594816/45929032 bytes (75.3%)

In [ ]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def drow_the_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=10)

    img = cv2.addWeighted(img, 1, blank_image, 1, 0.0)
    return img, blank_image



In [ ]:
def paint (img,vertices):
  mask = np.zeros_like(img)
  channel_count = img.shape[2]
  cv2.fillPoly(mask, vertices, color=(0, 255, 255))
  return mask

In [ ]:
def get_car_position(x_min,w):
    xm_per_pix=3.7/720
    if x_min is not None:
        car_position = w/2
        lane_center_position = (x_min+(700/2))
        center_dist = (car_position - lane_center_position) * xm_per_pix
    
    return center_dist 
def get_direction(center_dist):
    direction = ''
    if center_dist > 0:
        direction = 'right'
    elif center_dist < 0:
        direction = 'left'
    return direction

In [ ]:
def write_on_image (img,x_min) :
  font = cv2.FONT_HERSHEY_COMPLEX_SMALL
  text1=get_car_position(x_min,1280)
  direction=get_direction(text1)
  text=abs(get_car_position(x_min,1280))
  text = '{:04.3f}'.format(text) + 'm ' + direction + ' of center'
  edited_photo=cv2.putText(img, text,(40,70), cv2.FONT_HERSHEY_COMPLEX_SMALL,1.5,(255,255,255),2,cv2.LINE_AA)
  return edited_photo;

### phase 2

### Load weights and cfg

In [ ]:
weights_path =os.path.join("yolov3.weights")
cfg_path =os.path.join("yolov3.cfg.txt")

### load neural net in cv2 

In [ ]:
net=cv2.dnn.readNetFromDarknet(cfg_path,weights_path)

In [ ]:
names=net.getLayerNames() #get layers name

In [ ]:
layers_names=[names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
labels_path=os.path.join ("coco.names.txt")
labels = open(labels_path).read().strip().split("\n")

In [ ]:
def process(image):
  #print(image.shape)
  region_of_interest_vertices = [
    (200, 665),
    (620,410),
    (1130, 665)
  ]
  hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
  #h_channel=hls[:,:,0]
  #l_channel=hls[:,:,1]
  s_channel=hls[:,:,2]
  canny_image = cv2.Canny(s_channel, 100, 200)
  cropped_image = region_of_interest(canny_image,
                np.array([region_of_interest_vertices], np.int32))

  lines = cv2.HoughLinesP(cropped_image,
                        rho=6,
                        theta=np.pi/180,
                        threshold=160,
                        lines=np.array([]),
                        minLineLength=40,
                        maxLineGap=25)
  try :
    image_with_lines , mask = drow_the_lines(image, lines)
    c = (0,255,0)
    indices = np.where(np.all(mask == c, axis=-1))
    x_min=np.min(indices[1])
    if x_min<285 or x_min is None :
      x_min=285
    y_max=np.max(indices[0])
    x_topleft=x_min+310
    y_min=np.min(indices[0])
    if y_min<450 or y_min is None :
      y_min=450
  except:
      image_with_lines=image
      x_min=285
      x_topleft=660
      y_min=500
      y_max=670
  vertices_paint = [
    (x_min,y_max),
    (x_topleft,y_min),
    (x_topleft+45,y_min),
    (x_min+720, y_max)
  ]
  yellow_rec=paint(image,np.array([vertices_paint], np.int32))
  last_image = cv2.addWeighted(image_with_lines, 0.8, yellow_rec, 1, 0.0)
  last_edited=write_on_image (last_image,x_min)
  blob= cv2.dnn.blobFromImage(last_edited,1/255.0,(416,416),crop=False,swapRB=False)
  net.setInput(blob)
  layers_output=net.forward(layers_names)
  (H,W)=last_edited.shape[:2]
  boxes=[]
  confidences=[]
  classIDs=[]
  for output in layers_output:
    for detection in output:
        scores=detection[5:]
        classID=np.argmax(scores)
        confidence=scores[classID]
        
        if (confidence >0.85):
            box=detection[:4] * np.array([W,H,W,H])
            bx,by,bw,bh=box.astype("int")
            
            x=int(bx-(bw/2))
            y=int(by-(bh/2))
           
            boxes.append([x,y,int(bw),int(bh)])
            confidences.append(float(confidence))
            classIDs.append(classID)
    idx=cv2.dnn.NMSBoxes(boxes,confidences,0.8,0.8)
    if (len(idx)>0):
        for i in idx.flatten():
            (x,y)=[boxes[i][0],boxes[i][1]]
            (w,h)=[boxes[i][2],boxes[i][3]]
        cv2.rectangle (last_edited,(x,y),(x+w ,y+h),(0,139,139),2)
        cv2.putText(last_edited,"{}:{}".format(labels[classIDs[i]],confidences[i]),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,139,139),2)
  return last_edited


In [16]:

white_output = ("challenge_video_merged_full.mp4")
clip1 = VideoFileClip("challenge_video.mp4") 
white_clip = clip1.fl_image(process)
white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video challenge_video_merged_full.mp4
[MoviePy] Writing video challenge_video_merged_full.mp4


100%|██████████| 485/485 [10:58<00:00,  1.36s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_merged_full.mp4 

